![DoctorBert](https://snag.gy/oF84Gk.jpg)


![DoctorBert](https://i.ytimg.com/vi/nPemP-Q0Xn8/hqdefault.jpg)

This is a Colab Demo of our DocProduct Tensorflow 2.0 Hackathon Project

Project details can be seen on our Github repo
https://github.com/Santosh-Gupta/DocProduct

and our Devpost page
https://devpost.com/software/nlp-doctor

In order to fit the computational limits of Google Colab, we had to reduce the amount of searchable Q&A datapoints from ~700,000 to ~200,000.
We also converted our search embeddings from float32 to float16, although this does not seem to dimish the quality of search (*but that's a whole other product to explore*)

Instructions:

-Run the first cell to install TF2.0 and FAISS

-Run the second cell to download the models and data

-Run the third cell to load the weights and data

-Input your question and search parameters into the fourth cell to play around with our model results




In [1]:
#@title Install Faiss, TF 2.0, and our Github. Double Click to see code

#To use CPU FAISS use
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#To use GPU FAISS use
# !wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
!pip install https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
!pip install gpt2-estimator
!pip install pyarrow


--2019-09-12 09:49:50--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24, 2606:4700::6811:5c18, ...
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20190912T094950Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=AgoJb3JpZ2luX2VjEDAaCXVzLWVhc3QtMSJGMEQCID1kreB0ZfTrn1MnseKVjaQN3LpLG8CiknVCGJJ%2BFG1JAiBTkmTMTbXELiucm%2BctXmZtP1ZkfPJpsJA%2FYTERPvwmFSrjAwjp%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAAaDDQ1NTg2NDA5ODM3O

In [0]:
#@title Downaload all model checkpoints, and question/answer data. Double click to see this code

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
import os
import requests

import urllib.request

# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

file_id = '1uCXv6mQkFfpw5txGnVCsl93Db7t5Z2mp'

download_file_from_google_drive(file_id, 'Float16EmbeddingsExpanded5-27-19.pkl')

file_id = 'https://onedrive.live.com/download?cid=9DEDF3C1E2D7E77F&resid=9DEDF3C1E2D7E77F%2132792&authkey=AEQ8GtkcDbe3K98'
    
urllib.request.urlretrieve( file_id, 'DataAndCheckpoint.zip')

if not os.path.exists('newFolder'):
    os.makedirs('newFolder')

import zipfile
zip_ref = zipfile.ZipFile('DataAndCheckpoint.zip', 'r')
zip_ref.extractall('newFolder')
zip_ref.close()

In [0]:
#@title Load model weights and Q&A data. Double click to see code

from docproduct.predictor import RetreiveQADoc

pretrained_path = 'BioBertFolder/biobert_v1.0_pubmed_pmc/'
# ffn_weight_file = None
bert_ffn_weight_file = 'newFolder/models/bertffn_crossentropy/bertffn'
embedding_file = 'Float16EmbeddingsExpanded5-27-19.pkl'

doc = RetreiveQADoc(pretrained_path=pretrained_path,
ffn_weight_file=None,
bert_ffn_weight_file=bert_ffn_weight_file,
embedding_file=embedding_file)

In [15]:
#@title Type in your question (512 word limit) and search search parameters

question_text = "I have some vomiting sensations"  #@param {type:"string"}

search_similarity_by = 'answer'  #@param ['answer', "question"]

number_results_toReturn=10 #@param {type:"number"}

answer_only=True #@param ["False", "True"] {type:"raw"}

returned_results = doc.predict( question_text ,
                  search_by=search_similarity_by, topk=number_results_toReturn, answer_only=answer_only)

print('')
for jk in range(len(returned_results)):
    print("Result ", jk+1)
    print(returned_results[jk])
    print('')


1it [00:01,  1.58s/it]


Result  1
the possibility in my opinion is hiatal hernia causing acid reflux in your esophagus and this alleviates nausea. weak abdominal muscles and so diaphragm weakness can increase the possibility of the stomach to bulge upwards through your diaphragm to the thtiacic cavity and so causing nausea also. an new endoscopy and repair of the hiatsl hernia can be the solution of your problem. consider deep diaphragmatic breathing exercise to strength your diaphragm.

Result  2
hellono i haven’t been vomiting nor i had diarrhea just very uncomfortable stomach and stiff muscles all over neck specially.

Result  3
nausea vomiting headache and impairment of level of consciousness!

Result  4
Have you had any sort of imaging done? You could have an abnormality in your esophagus or stomach that’s causing your morning nausea/vomiting. There’s no way to know for sure without getting scanned, but it might be worth it. A hiatal hernia is an outpouching of part of the stomach upwards through the di